In [ ]:
# ==============================================================================
# SEÇÃO 0: SETUP E INICIALIZAÇÃO
# ==============================================================================
# Este bloco configura o ambiente, importa as bibliotecas necessárias e
# inicializa o AnalisadorBancario, deixando-o pronto para uso.
# ------------------------------------------------------------------------------
print("--- INICIANDO SETUP E INICIALIZAÇÃO ---")

import pandas as pd
import numpy as np
from pathlib import Path
import sys
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker

# Adiciona o diretório 'Code' ao path para encontrar o DataUtils
code_dir = Path('..').resolve() / 'Code'
if str(code_dir) not in sys.path:
    sys.path.append(str(code_dir))

from DataUtils import AnalisadorBancario

# Configurações de exibição do Pandas e Matplotlib
pd.set_option('display.float_format', lambda x: f'{x:,.2f}')
pd.set_option('display.max_rows', 100)
plt.style.use('seaborn-v0_8-whitegrid')

# Inicializa o Analisador (carrega todos os dados em memória)
output_dir = Path('..').resolve() / 'Output'
analisador = AnalisadorBancario(diretorio_output=str(output_dir))

print("\n--- SETUP CONCLUÍDO. ANALISADOR PRONTO PARA USO. ---")

In [ ]:
# ==============================================================================
# SEÇÃO 1: CONSULTAS FUNDAMENTAIS
# ==============================================================================
# Demonstração dos métodos básicos para extrair dados de cada fonte.
# ------------------------------------------------------------------------------

# --- 1.1: Buscando dados COSIF (por documento específico) ---
print("\n--- 1.1: Consulta de dados COSIF (Prudencial) ---")
# A função determina o tipo (prudencial) automaticamente pelo documento 4060.
dados_cosif = analisador.get_dados_cosif(
    identificador='00000000', # CNPJ do Banco do Brasil
    contas=['ATIVO TOTAL', 'PATRIMONIO LIQUIDO'],
    datas=202403,
    documentos=4060
)
print("Resultado para 'BANCO DO BRASIL S.A.' (COSIF Prudencial):")
display(dados_cosif)
print("-" * 80)


# --- 1.2: Buscando dados IF.DATA (com controle de escopo) ---
print("\n--- 1.2: Consulta de dados IF.DATA (Escopo Individual) ---")
# O novo parâmetro `escopo` permite especificar exatamente qual dado buscar.
dados_ifdata = analisador.get_dados_ifdata(
    identificador='60701190', # CNPJ do Itaú Unibanco
    contas=['Índice de Basileia', 'Lucro Líquido'],
    datas=202403,
    escopo='individual' # Buscando o dado da instituição, não do conglomerado.
)
print("Resultado para 'ITAÚ UNIBANCO S.A.' (IF.DATA Individual):")
display(dados_ifdata)
print("-" * 80)


# --- 1.3: Buscando atributos do cadastro ---
print("\n--- 1.3: Consulta de atributos de cadastro de Múltiplos Bancos ---")
atributos_cadastro = analisador.get_atributos_cadastro(
    identificador=['60701190', '00360305', '30306294'], # Itaú, Caixa, BTG
    atributos=['TCB_IFD_CAD', 'SEGMENTOTB_IFD_CAD', 'DATA_INICIO_ATIVIDADE_IFD_CAD']
)
print("Resultado para atributos de cadastro:")
display(atributos_cadastro)
print("-" * 80)

In [ ]:
# ==============================================================================
# SEÇÃO 2: ANÁLISE E VISUALIZAÇÃO
# ==============================================================================
# Demonstração dos métodos de alto nível para análises comparativas e temporais.
# ------------------------------------------------------------------------------

# --- 2.1: Tabela Comparativa de Pares ---
print("\n--- 2.1: Tabela comparativa de indicadores (Pares) ---")
# Combinando fontes e escopos diferentes em uma única chamada.
indicadores = {
    'Patrimônio Líquido': {'tipo': 'COSIF', 'conta': 'PATRIMONIO LIQUIDO', 'documento_cosif': 4010},
    'Ativo Total': {'tipo': 'IFDATA', 'conta': 'Ativo Total', 'escopo_ifdata': 'prudencial'},
    'Índice de Basileia': {'tipo': 'IFDATA', 'conta': 'Índice de Basileia', 'escopo_ifdata': 'prudencial'},
    'Segmento': {'tipo': 'ATRIBUTO', 'atributo': 'SEGMENTOTB_IFD_CAD'}
}
bancos = ['BANCO DO BRASIL S.A.', 'BANCO BRADESCO S.A.', '60701190', 'BTG Pactual']

tabela_comparativa = analisador.comparar_indicadores(
    identificadores=bancos,
    indicadores=indicadores,
    data=202403
)
print("Comparativo de indicadores para os principais bancos:")
display(tabela_comparativa)
print("-" * 80)


# --- 2.2: Série Temporal e Visualização Gráfica ---
print("\n--- 2.2: Série temporal de um indicador e sua visualização ---")
# Usando os novos parâmetros para manter NaNs na série e visualizar as lacunas.
serie_lucro_bbtg = analisador.get_serie_temporal_indicador(
    identificador='BTG Pactual',
    conta='Lucro Líquido',
    data_inicio=202201,
    data_fim=202312,
    fonte='IFDATA',
    escopo_ifdata='prudencial', # Buscando o lucro do conglomerado
    drop_na=False # Mantém os meses sem dados como NaN para visualização
)
print("Série Temporal para 'Lucro Líquido' do BTG Pactual:")
display(serie_lucro_bbtg)

# Plotando o resultado para visualizar a sazonalidade (dados trimestrais)
if not serie_lucro_bbtg.empty:
    fig, ax = plt.subplots(figsize=(12, 6))
    serie_lucro_bbtg.plot(x='DATA', y='Valor', ax=ax, marker='o', linestyle='-', 
                          title="Evolução do Lucro Líquido - BTG Pactual (Prudencial)")
    ax.set_ylabel("Valor (em R$)")
    ax.set_xlabel("Data")
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda x, p: f'R$ {x/1e9:,.1f} bi'))
    plt.tight_layout()
    plt.show()
print("-" * 80)

In [ ]:
# ==============================================================================
# SEÇÃO 3: TÉCNICAS AVANÇADAS E CONTROLE DE DADOS
# ==============================================================================
# Demonstração da flexibilidade da classe em cenários complexos.
# ------------------------------------------------------------------------------

# --- 3.1: Comparando Escopos no IF.DATA ---
print("\n--- 3.1: Diferença entre escopos de dados no IF.DATA ---")
conta_imob = 'Ativo Total'
data_imob = 202312
id_bradesco = 'BANCO BRADESCO S.A.'

# Usando o novo parâmetro `escopo` para comparar os mesmos dados
dado_ifd_prudencial = analisador.get_dados_ifdata(identificador=id_bradesco, contas=[conta_imob], datas=data_imob, escopo='prudencial')
dado_ifd_individual = analisador.get_dados_ifdata(identificador=id_bradesco, contas=[conta_imob], datas=data_imob, escopo='individual')

val_prud = dado_ifd_prudencial['VALOR_CONTA_IFD_VAL'].iloc[0] if not dado_ifd_prudencial.empty else "N/A"
val_ind = dado_ifd_individual['VALOR_CONTA_IFD_VAL'].iloc[0] if not dado_ifd_individual.empty else "N/A"

print(f"Ativo Total (Prudencial) do Bradesco: {val_prud}")
print(f"Ativo Total (Individual) do Bradesco: {val_ind}")
print("-" * 80)


# --- 3.2: Controle Explícito de Dados Ausentes e Zeros ---
print("\n--- 3.2: Flexibilidade no tratamento de nulos e zeros ---")
banco_serie = 'Banco Inter'
conta_serie = 'Instrumentos Derivativos (f)' # Uma conta que pode ter valores zero
data_inicio, data_fim = 202301, 202306

print("(A) Comportamento Padrão (remove linhas com NaN):")
serie_padrao = analisador.get_serie_temporal_indicador(
    identificador=banco_serie, conta=conta_serie, fonte='IFDATA', escopo_ifdata='individual',
    data_inicio=data_inicio, data_fim=data_fim, drop_na=True
)
display(serie_padrao)

print("\n(B) Preenchendo dados ausentes (NaN) com 0:")
serie_fill_zero = analisador.get_serie_temporal_indicador(
    identificador=banco_serie, conta=conta_serie, fonte='IFDATA', escopo_ifdata='individual',
    data_inicio=data_inicio, data_fim=data_fim, fill_value=0, drop_na=False
)
display(serie_fill_zero)

print("\n(C) Removendo Zeros da série (tratando-os como dados ausentes):")
serie_zeros_nan = analisador.get_serie_temporal_indicador(
    identificador=banco_serie, conta=conta_serie, fonte='IFDATA', escopo_ifdata='individual',
    data_inicio=data_inicio, data_fim=data_fim, replace_zeros_with_nan=True, drop_na=True
)
display(serie_zeros_nan)
print("-" * 80)


# --- 3.3: Robustez Contra Inputs Inválidos ---
print("\n--- 3.3: Tratamento de inputs inválidos ---")
# A classe deve retornar avisos e DataFrames vazios ou com N/A, sem quebrar.
tabela_invalida = analisador.comparar_indicadores(
    identificadores=['BANCO QUE NAO EXISTE', '99999999'],
    indicadores={'Indicador Inválido': {'tipo': 'COSIF', 'conta': 'CONTA QUE NAO EXISTE'}},
    data=202403
)
print("Resultado da tabela com inputs inválidos (esperado: tabela com erros e Nones):")
display(tabela_invalida)
print("-" * 80)


print("\n\n--- FIM DAS DEMONSTRAÇÕES ---")